### 2. 下载 GROMACS 源代码

你可以从 GROMACS 官方网站下载 5.0.7 版本的源代码。使用以下命令：

```bash
wget ftp://ftp.gromacs.org/pub/gromacs/gromacs-5.0.7.tar.g
```


#### 1. 将smiles结构转换为zmat结构

In [75]:
from rdkit import Chem
from rdkit.Chem import AllChem

# SMILES 字符串示例
smiles = "CCN1C=C[N+](=C1)C"  # 例如，乙醇的 SMILES 字符串

# 从 SMILES 创建 RDKit 分子对象
molecule = Chem.MolFromSmiles(smiles)
molecule = Chem.AddHs(molecule)

# 生成 3D 坐标
AllChem.EmbedMolecule(molecule, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(molecule)

def write_xyz(mol, filename):
    num_atoms = mol.GetNumAtoms()
    conformer = mol.GetConformer()
    elements = [mol.GetAtomWithIdx(i).GetSymbol() for i in range(num_atoms)]

    with open(filename, 'w') as file:
        file.write(str(num_atoms) + '\n')
        file.write("XYZ file generated by RDKit\n")
        for i in range(num_atoms):
            pos = conformer.GetAtomPosition(i)
            line = "{} {:.4f} {:.4f} {:.4f}\n".format(elements[i], pos.x, pos.y, pos.z)
            file.write(line)

# 使用函数保存 XYZ 文件
write_xyz(molecule, "molecule.xyz")

#### 2. 建立离子液体GROMACS输入文件

In [1]:
# Use fftool to create an input file for packmol
!fftool 20 ./data/c4c1im.zmat 20 ./data/ntf2.zmat -r 3.0

density 3.000 mol/L  volume 22141.0 A^3
molecule_file      species           nmol force_field      nbond source  charge
  ./data/c4c1im.zmat c4c1im+             20 il.ff               25 file   +1.0000
  ./data/ntf2.zmat tf2N-               20 il.ff               14 file   -1.0000
packmol file
  pack.inp


In [2]:
# Use packmol with the pack.inp file
!packmol < pack.inp


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.2 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: xyz
  Seed for random number generator:      1234567
  Output file: simbox.xyz
  Reading coordinate file: ./data/c4c1im_pack.xyz
  Reading coordinate file: ./data/ntf2_pack.xyz
  Number of independent structures:            2
  The structures are: 
  Structure            1 :c4c1im+ il.ff(          25  atoms)
  Structure            2 :tf2N- il.ff(          15  atoms)
  Maximum number of GENCAN loops for all molecule packing:          400

          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .35464E+02
  Best function value before: f = .87346E+04
  Improvement from best function value:    99.59 %
  Improvement from last loop:    99.59 %
  Maximum violation of target distance:     1.037386
  Maximum violation of the constraints: .44539E+01

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:            1
  Scaling radii by:    1.1000000000000001     

  Packing:|0                                                             100%|
          |******************************************************************|
          |******************************************************************|

  Function value from last GENCAN loop: f = .61779E+01
 

In [3]:
# Use fftool to create input files for GROMACS (-g)
!fftool 20 ./data/c4c1im.zmat 20 ./data/ntf2.zmat -r 3.0 -g

density 3.000 mol/L  volume 22141.0 A^3
molecule_file      species           nmol force_field      nbond source  charge
  ./data/c4c1im.zmat c4c1im+             20 il.ff               25 file   +1.0000
  ./data/ntf2.zmat tf2N-               20 il.ff               14 file   -1.0000
gromacs files
  run.mdp
  field.top
  config.pdb


In [1]:
!export PATH=/usr/local/gromacs-5.0.7/bin:$PATH
!source /usr/local/gromacs-5.0.7/bin/GMXRC

In [3]:
!pip install GromacsWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 152.4 kB/s eta 0:00:00a 0:00:01


In [12]:
import gromacs
gromacs.config.setup()

NOTE: GromacsWrapper created the configuration file 
	'/Users/tanshendong/.gromacswrapper.cfg'
      for you. Edit the file to customize the package.


In [16]:
gromacs.editconf(f="config.pdb", o="conf.gro", d=1.5, c=True)
# !gmx editconf -f config.pdb -o conf.gro
gromacs.grompp(f="run.mdp", c="conf.gro", p="field.top", o="run.tpr")
# !gmx grompp -f run.mdp -c conf.gro -p field.top -o run

Read 800 atoms
Volume: 22.1407 nm^3, corresponds to roughly 9900 electrons
No velocities found
    system size :  2.520  2.522  2.519 (nm)
    center      :  1.538  1.485  1.421 (nm)
    box vectors :  2.808  2.808  2.808 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  22.14               (nm^3)
    shift       :  1.222  1.276  1.338 (nm)
new center      :  2.760  2.761  2.759 (nm)
new box vectors :  5.520  5.522  5.519 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  : 168.21               (nm^3)

If the molecule rotates the actual distance will be smaller. You might want
to use a cubic box instead, or why not try a dodecahedron today?
turning H bonds into constraints...
turning H bonds into constraints...
Analysing residue names:
There are:    40      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K
Calculated rlist 

GROMACS:    gmx editconf, VERSION 5.0.7

GROMACS is written by:
Emile Apol         Rossen Apostolov   Herman J.C. Berendsen Par Bjelkmar       
Aldert van Buuren  Rudi van Drunen    Anton Feenstra     Sebastian Fritsch  
Gerrit Groenhof    Christoph Junghans Peter Kasson       Carsten Kutzner    
Per Larsson        Justin A. Lemkul   Magnus Lundborg    Pieter Meulenhoff  
Erik Marklund      Teemu Murtola      Szilard Pall       Sander Pronk       
Roland Schulz      Alexey Shvetsov    Michael Shirts     Alfons Sijbers     
Peter Tieleman     Christian Wennberg Maarten Wolf       
and the project leaders:
Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2014, The GROMACS development team at
Uppsala University, Stockholm University and
the Royal Institute of Technology, Sweden.
check out http://www.gromacs.org for more information.

GROMACS is free software; you can redistribute it and/or

(0, None, None)